# Guided Capstone Step 4. Pre-Processing and Training Data Development

**The Data Science Method**  


1.   Problem Identification 


2.   Data Wrangling 
  
 
3.   Exploratory Data Analysis   

4.   **Pre-processing and Training Data Development**  
 * Create dummy or indicator features for categorical variables
  * Standardize the magnitude of numeric features
  * Split into testing and training datasets
  * Apply scaler to the testing set
5.   Modeling 
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

**<font color='teal'> Start by loading the necessary packages as we did in step 3 and printing out our current working directory just to confirm we are in the correct project directory. </font>**

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
%matplotlib inline
os.getcwd()

'/Users/ktregier/Desktop/Springboard/Unit 6/Capstone1/5.Modeling'

**<font color='teal'>  Load the csv file you created in step 3, remember it should be saved inside your data subfolder and print the first five rows.</font>**

In [2]:
filepath = '/Users/ktregier/Desktop/Springboard/Unit 6/Capstone1/3.EDA/step3.2_output.csv'
df = pd.read_csv(filepath, header=0, index_col=0)
df.sort_values('state')

,Name,state,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,clusters
0,Alyeska Resort,Alaska,3939,2500,1,0.0,0,2,2,0,...,1610.0,113.0,150.000000,60.0,669.0,65.000000,85.00000,150.000000,550.0,1
1,Eaglecrest Ski Area,Alaska,2600,1540,0,0.0,0,0,0,0,...,640.0,60.0,45.000000,44.0,350.0,47.000000,53.00000,90.000000,0.0,1
2,Hilltop Ski Area,Alaska,2090,294,0,0.0,0,0,0,1,...,30.0,30.0,150.000000,36.0,69.0,30.000000,34.00000,152.000000,30.0,1
3,Arizona Snowbowl,Arizona,11500,2300,0,0.0,1,0,2,2,...,777.0,104.0,122.000000,81.0,260.0,89.000000,89.00000,122.000000,0.0,2
4,Sunrise Park Resort,Arizona,11100,1800,0,0.0,0,1,2,3,...,800.0,80.0,115.000000,49.0,250.0,74.000000,78.00000,104.000000,80.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,Snowy Range Ski & Recreation Area,Wyoming,9663,990,0,0.0,0,0,0,1,...,75.0,30.0,131.000000,59.0,250.0,49.000000,49.00000,120.053004,0.0,2
323,Hogadon Basin,Wyoming,8000,640,0,0.0,0,0,0,0,...,92.0,32.0,121.000000,61.0,80.0,48.000000,48.00000,95.000000,0.0,2
322,Grand Targhee Resort,Wyoming,9920,2270,0,0.0,0,2,2,0,...,2602.0,0.0,152.000000,50.0,500.0,90.000000,90.00000,152.000000,0.0,2
324,Jackson Hole,Wyoming,10450,4139,3,0.0,0,4,4,1,...,2500.0,195.0,130.000000,54.0,459.0,57.916957,64.16681,133.000000,0.0,2


## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `state`. Add the dummies back to the dataframe and remove the original column for `state`. </font>**

Hint: you can see an example of how to execute this in Aiden's article on preprocessing [here](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-pre-processing-and-training-data-development-fd2d75182967). 

In [3]:
#dfo = df.select_dtypes(include=['object'])
df = pd.concat([df.drop ('state', axis=1), pd.get_dummies(df.state)], axis=1)

## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing standardize the scale of the features of the dataframe except the name of the resort which we done't need in the dataframe for modeling, so it can be droppped here as well. Also, we want to hold out our response variable(s) so we can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as our response for scaling and modeling. Later we will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave them in the development dataframe. </font>**

In [4]:
# first we import the preprocessing package from the sklearn library
from sklearn import preprocessing

# Declare an explanatory variable, called X,and assign it the result of dropping 'Name' and 'AdultWeekend' from the df
X = df.drop(['Name','AdultWeekend'], axis=1)

# Declare a response variable, called y, and assign it the AdultWeekend column of the df 
y = df.AdultWeekend 

# Here we use the StandardScaler() method of the preprocessing package, and then call the fit() method with parameter X 
scaler = preprocessing.StandardScaler().fit(X)

# Declare a variable called X_scaled, and assign it the result of calling the transform() method with parameter X 
X_scaled=scaler.transform(X)

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selection import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. We will start by using the adult weekend ticket price as our response variable for modeling.</font>**

In [5]:
# Import the train_test_split function from the sklearn.model_selection utility.  
from sklearn.model_selection import train_test_split

# Get the 1-dimensional flattened array of our response variable y by calling the ravel() function on y
y = y.ravel()

# Call the train_test_split() function with the first two parameters set to X_scaled and y 
# Declare four variables, X_train, X_test, y_train and y_test separated by commas 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=1)

Here we start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5. Modeling


This is the fifth step in the Data Science Method. In the previous steps you cleaned and prepared the datasets. Now it's time to get into the most exciting part: modeling! In this exercise, you'll build three different models and compare each model's performance. In the end, you'll choose the best model for demonstrating insights to Big Mountain management.



### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with a Training Dataset

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [6]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import explained_variance_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

Hint: you will have to use the `predict()` method here as it's used in this [DSM article](https://medium.com/@aiden.dataminer/the-data-science-method-dsm-modeling-56b4233cad1b) about modeling. 

In [7]:
# Make a variable called y_pred and assign it the result of calling predict() on our model variable with parameter X_test
y_pred = model.predict(X_test)

## Review Model Outcomes — Iterate over additional models as needed

In [8]:
# You might want to use the explained_variance_score() and mean_absolute_error() metrics.
# To do so, you will need to import them from sklearn.metrics. 
# You can plug y_test and y_pred into the functions to evaluate the model

In [9]:
from sklearn.metrics import mean_absolute_error, explained_variance_score
from math import sqrt
mean_absolute_error(y_test, y_pred)

4.858650645442623

In [10]:
print("ESV = ")
explained_variance_score( y_test, y_pred)

ESV = 


0.9411044866574266

**<font color='teal'> Print the intercept value from the linear model. </font>**

Hint: our linear regression model `lm` has an attribute `intercept_` for the intercept

In [11]:
print(lm.intercept_)

64.09476880620056


**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the resulting `AdultWeekend` value. Also, because we took the time to scale our x values in the training data, we can compare each of the coeeficients for the features to determine the feature importances. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 


Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what we are looking for is the magnitude of impact on our response variable.

In [12]:
# You might want to make a pandas DataFrame displaying the coefficients for each state like so: 
result = pd.DataFrame(abs(lm.coef_), X.columns, columns=['Coefficient'])
result.sort_values('Coefficient' , ascending = False)
#result.head(25)

,Coefficient
total_chairs,3.282331e+13
fastQuads,1.244339e+13
surface,1.165852e+13
double,1.027392e+13
triple,9.165045e+12
quad,7.427929e+12
New York,6.049477e+12
Michigan,5.709064e+12
Colorado,5.030016e+12
California,4.922339e+12


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that we care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

Hint: Try to construct another model using exactly the steps we followed above. 

#### Model 2

In [13]:
model2 = pd.read_csv(filepath, header=0, index_col=0)
model2X = model2.drop(['AdultWeekend', 'Name', 'state'], axis=1)
model2y = model2.AdultWeekend
scaler = preprocessing.StandardScaler().fit(model2X)
X2_scaled=scaler.transform(model2X)
y2 = model2y.ravel()
X2_train, X2_test, y2_train, y2_test = train_test_split(X2_scaled, y2, test_size=0.25, random_state=1)
lm2 = linear_model.LinearRegression()
m2 = lm2.fit(X2_train, y2_train)
y2_pred = m2.predict(X2_test)
print("ESV: ", explained_variance_score(y2_test, y2_pred))
print("MAE: " , mean_absolute_error(y2_test, y2_pred))
print("Intercept: ", lm2.intercept_)

ESV:  0.9339850010425124
MAE:  5.22613139183815
Intercept:  64.0704745726884


In [14]:
result2 = pd.DataFrame(abs(lm2.coef_), model2X.columns, columns=['Coefficient'])
result2.sort_values('Coefficient' , ascending = False)

,Coefficient
AdultWeekday,19.746480
clusters,5.537925
summit_elev,4.380184
SkiableTerrain_ac,1.894804
surface,1.459595
triple,1.409946
Runs,1.292027
fastQuads,1.193568
vertical_drop,1.168133
quad,1.049086


**<font color='teal'> When reviewing our new model coefficients, we see `summit_elev` is now in the number two spot. This is also difficult to change from a management prespective and highly correlated with `base_elev` and `vertical_drop`.  This time, rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [15]:
model3 = pd.read_csv(filepath, header=0, index_col=0)
model3.columns
model3X = model3.drop(['AdultWeekend', 'Name', 'state', 'summit_elev'], axis=1)
model3y = model3.AdultWeekend
scaler = preprocessing.StandardScaler().fit(model3X)
X3_scaled=scaler.transform(model3X)
y3 = model3y.ravel()
X3_train, X3_test, y3_train, y3_test = train_test_split(X3_scaled, y3, test_size=0.25, random_state=1)
lm3 = linear_model.LinearRegression()
m3 = lm3.fit(X3_train, y3_train)
y3_pred = m3.predict(X3_test)
print("ESV: ", explained_variance_score(y3_test, y3_pred))
print("MAE: ", mean_absolute_error(y3_test, y3_pred))
print("Intercept: ", lm3.intercept_)

ESV:  0.9358283640758421
MAE:  5.151355300611119
Intercept:  64.06696562448211


In [16]:
result3 = pd.DataFrame(abs(lm3.coef_), model3X.columns, columns=['Coefficient'])
result3.sort_values('Coefficient' , ascending = False)

,Coefficient
AdultWeekday,20.025346
clusters,2.637841
vertical_drop,2.113019
triple,1.404602
surface,1.267252
daysOpenLastYear,1.227273
quad,1.152997
Runs,1.113634
SkiableTerrain_ac,0.939377
fastQuads,0.812288


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell — you will discuss this selection more in the next step of the guided casptone. </font>**

| Model | Explained Variance| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | 0.9411044866574266 | 4.858650645442623 |-|
| Model 2. | 0.9339850010425124| 5.22613139183815 |'state'|
| Model 3. | 0.9358283640758421 | 5.151355300611119 |'state','summit_elev','base_elev'|

Model Selection:

The intercept value for all three models is similar.

I thought it was interesting that the model with the best performance (ss I understand it) is model 1. I guess it indicates that non-changable features (location and elevation) have an effect on ticket price.

I would probably choose model 3 for the basis of my presentation. It explains slightly more variation with slightly less error than model 2, and doesn't include the unchangeable factors in model 1.